<a href="https://colab.research.google.com/github/palak-38/inter_iit_trials/blob/main/gradcam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
model=models.resnet50(pretrained=True)
model.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 118MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
final_conv_layer = model.layer4[-1].conv2

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    return transform(image).unsqueeze(0)

In [ ]:
def forward_hook(module, input, output):
    global feature_maps
    feature_maps = output

In [ ]:
def backward_hook(module, grad_input, grad_output):
    global gradients
    gradients = grad_output[0]

In [ ]:
def backward_hook(module, grad_input, grad_output):
    global gradients
    gradients = grad_output[0]

In [ ]:
def generate_gradcam(model, image_tensor, target_class):
    # Forward pass
    output = model(image_tensor)

    # Set the model in evaluation mode and get predictions
    model.zero_grad()
    class_score = output[:, target_class]

    # Backward pass to get the gradient of the target class
    class_score.backward()

    # Compute the weights (global average pooling over gradients)
    alpha = torch.mean(gradients, dim=[0, 2, 3])

    # Compute Grad-CAM map as weighted sum of feature maps
    gradcam_map = torch.zeros(feature_maps.shape[2:], dtype=torch.float32)
    for i in range(len(alpha)):
        gradcam_map += alpha[i] * feature_maps[0, i, :, :]

    # Apply ReLU to the Grad-CAM map
    gradcam_map = F.relu(gradcam_map)

    # Normalize the heatmap to [0, 1]
    gradcam_map -= gradcam_map.min()
    gradcam_map /= gradcam_map.max()

    return gradcam_map.detach().numpy()

In [ ]:
def overlay_heatmap_on_image(heatmap, original_image_path):
    heatmap = np.uint8(255 * heatmap)
    heatmap = np.array(Image.fromarray(heatmap).resize((224, 224)))

    original_image = Image.open(original_image_path).convert('RGB')
    original_image = np.array(original_image.resize((224, 224)))
    overlay = np.uint8(original_image * 0.6 + plt.cm.jet(heatmap)[:, :, :3] * 0.4 * 255)

    return overlay

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving catt.jpg to catt (1).jpg


In [ ]:
if __name__ == "__main__":
    image_path = "catt.jpg"  # Replace with your image path
    image_tensor = load_image(image_path)

    # Predict the target class (here using argmax for simplicity)
    output = model(image_tensor)
    target_class = output.argmax().item()

    # Generate Grad-CAM heatmap
    gradcam_heatmap = generate_gradcam(model, image_tensor, target_class)

    # Overlay heatmap and display
    overlay = overlay_heatmap_on_image(gradcam_heatmap, image_path)
    plt.imshow(overlay)
    plt.axis('off')
    plt.show()


NameError: name 'gradients' is not defined